# TPS Dec 2021 — Plan and Experiment Log

Goal: WIN A MEDAL (>= 0.95658 accuracy on LB).

Approach:
- Baseline: LightGBM multiclass with stratified 5-fold CV, robust seed control, early stopping.
- Features: start with raw features; handle Id if present; map target to 0-based for LGBM then back.
- Validation: 5-fold StratifiedKFold; track per-fold accuracy and OOF accuracy.
- Inference: average fold probs; argmax; write submission.csv.

Next Steps:
1) Load data, inspect shapes and target distribution.
2) Train LGBM baseline; log fold timings and accuracy.
3) If CV < 0.955, iterate: tune num_leaves, max_depth, feature_fraction, learning_rate; try more folds, seeds, or XGBoost/CatBoost, and simple feature interactions.
4) Ensembling/seeds if needed to push over medal threshold.

Experiment Log:
- [TBD] Exp01: LGBM 5F baseline.

In [2]:
import time, pandas as pd
print("Sanity check: starting quick I/O test...", flush=True)
t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(f"Loaded. train: {train.shape}, test: {test.shape}", flush=True)
print("Train columns (first 10):", list(train.columns[:10]))
print("Target value_counts (top):\n", train['Cover_Type'].value_counts().sort_index().head(10))
print(f"Done quick I/O test in {time.time()-t0:.2f}s", flush=True)

Sanity check: starting quick I/O test...


Loaded. train: (3600000, 56), test: (400000, 55)


Train columns (first 10): ['Id', 'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
Target value_counts (top):
 Cover_Type
1    1320866
2    2036254
3     176184
4        333
5          1
6      10237
7      56125
Name: count, dtype: int64
Done quick I/O test in 12.00s


In [12]:
import sys, subprocess, time, os, math, traceback, gc
t0_all = time.time()

def ensure(pkg):
    try:
        __import__(pkg)
        print(f"[ok] {pkg} already installed")
    except ImportError:
        print(f"[install] {pkg} ...", flush=True)
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])
        __import__(pkg)

# Ensure key packages
ensure('numpy'); ensure('pandas'); ensure('lightgbm')
import numpy as np, pandas as pd, lightgbm as lgb

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, KFold
from sklearn.metrics import accuracy_score

def downcast_df(df: pd.DataFrame) -> pd.DataFrame:
    for c in df.columns:
        if pd.api.types.is_integer_dtype(df[c]):
            df[c] = pd.to_numeric(df[c], downcast='integer')
        elif pd.api.types.is_float_dtype(df[c]):
            df[c] = pd.to_numeric(df[c], downcast='float')
    return df

def fast_onehot_to_cat(df: pd.DataFrame, prefix: str, start_at_one: bool = True):
    cols = [c for c in df.columns if c.startswith(prefix)]
    if not cols:
        return None, None
    # Sort by numeric suffix to ensure correct order
    def suf(c):
        try:
            return int(c.split('_')[-1])
        except Exception:
            return c
    cols = sorted(cols, key=suf)
    arr = df[cols].to_numpy(copy=False)
    if arr.dtype != np.int8:
        arr = arr.astype(np.int8, copy=False)
    cat = arr.argmax(axis=1).astype(np.int16)
    if start_at_one:
        cat = (cat + 1).astype(np.int16)
    cat_sum = arr.sum(axis=1).astype(np.int16)
    df.drop(columns=cols, inplace=True)
    return cat, cat_sum

def add_features(df: pd.DataFrame) -> pd.DataFrame:
    Hhyd = 'Horizontal_Distance_To_Hydrology'
    Vhyd = 'Vertical_Distance_To_Hydrology'
    elev = 'Elevation'
    hs9, hs12, hs3 = 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm'
    aspect = 'Aspect'
    Hroad = 'Horizontal_Distance_To_Roadways'
    Hfire = 'Horizontal_Distance_To_Fire_Points'

    # Core engineered features
    if all(col in df.columns for col in [Hhyd, Vhyd]):
        df['Euclidean_Distance_To_Hydrology'] = np.sqrt(df[Hhyd]**2 + df[Vhyd]**2)
        df['Manhattan_Distance_To_Hydrology'] = np.abs(df[Hhyd]) + np.abs(df[Vhyd])
        df['Elevation_VD_Hydrology'] = df['Elevation'] - df[Vhyd]
        df['Elevation_Plus_VD_Hydrology'] = df['Elevation'] + df[Vhyd]
        # Extra FE per expert
        df['Elevation_minus_Euclidean_Dist_Hydrology'] = df['Elevation'] - df['Euclidean_Distance_To_Hydrology']
        df['Hydro_Ratio'] = df[Hhyd] / (df[Vhyd].abs() + 1.0)
        # Hydrology angle features
        ang = np.arctan2(df[Vhyd].astype(float), df[Hhyd].astype(float))
        df['Hydro_Angle_Sin'] = np.sin(ang)
        df['Hydro_Angle_Cos'] = np.cos(ang)
    else:
        df['Euclidean_Distance_To_Hydrology'] = 0.0
        df['Manhattan_Distance_To_Hydrology'] = 0.0
        df['Elevation_VD_Hydrology'] = 0.0
        df['Elevation_Plus_VD_Hydrology'] = 0.0
        df['Elevation_minus_Euclidean_Dist_Hydrology'] = 0.0
        df['Hydro_Ratio'] = 0.0
        df['Hydro_Angle_Sin'] = 0.0
        df['Hydro_Angle_Cos'] = 0.0

    if all(col in df.columns for col in [hs9, hs12, hs3]):
        df['Hillshade_Mean'] = (df[hs9] + df[hs12] + df[hs3]) / 3.0
        df['Hillshade_Min'] = df[[hs9, hs12, hs3]].min(axis=1)
        df['Hillshade_Max'] = df[[hs9, hs12, hs3]].max(axis=1)
        df['Hillshade_Range'] = df['Hillshade_Max'] - df['Hillshade_Min']
        df['Hillshade_Diff_9_3'] = df[hs9] - df[hs3]
    else:
        df['Hillshade_Mean'] = 0.0
        df['Hillshade_Min'] = 0.0
        df['Hillshade_Max'] = 0.0
        df['Hillshade_Range'] = 0.0
        df['Hillshade_Diff_9_3'] = 0.0

    if aspect in df.columns:
        rad = np.deg2rad(df[aspect].astype(float))
        df['Aspect_Sin'] = np.sin(rad)
        df['Aspect_Cos'] = np.cos(rad)
    else:
        df['Aspect_Sin'] = 0.0
        df['Aspect_Cos'] = 0.0

    # Distance interactions
    if Hroad in df.columns and Hfire in df.columns:
        df['Road_Fire_AbsDiff'] = np.abs(df[Hroad] - df[Hfire])
    else:
        df['Road_Fire_AbsDiff'] = 0.0
    if Hhyd in df.columns and Hroad in df.columns:
        df['Hydro_Road_AbsDiff'] = np.abs(df[Hhyd] - df[Hroad])
    else:
        df['Hydro_Road_AbsDiff'] = 0.0
    if Hhyd in df.columns and Hfire in df.columns:
        df['Hydro_Fire_AbsDiff'] = np.abs(df[Hhyd] - df[Hfire])
    else:
        df['Hydro_Fire_AbsDiff'] = 0.0

    # Compress Wilderness_Area and Soil_Type
    w_cat, w_sum = fast_onehot_to_cat(df, 'Wilderness_Area', start_at_one=True)
    if w_cat is not None:
        df['Wilderness_Area_cat'] = w_cat
        df['Wilderness_Area_Sum'] = w_sum
    else:
        df['Wilderness_Area_cat'] = 0
        df['Wilderness_Area_Sum'] = 0
    s_cat, s_sum = fast_onehot_to_cat(df, 'Soil_Type', start_at_one=True)
    if s_cat is not None:
        df['Soil_Type_cat'] = s_cat
        df['Soil_Type_Sum'] = s_sum
    else:
        df['Soil_Type_cat'] = 0
        df['Soil_Type_Sum'] = 0

    # Additional categorical interaction
    df['Soil_Wilderness_Interaction'] = (df['Soil_Type_cat'].astype(np.int32) * 100 + df['Wilderness_Area_cat'].astype(np.int32)).astype(np.int32)
    # Elevation binned (coarse)
    try:
        df['Elevation_Binned'] = pd.cut(df['Elevation'], bins=30, labels=False).astype('float32').fillna(-1).astype('int16')
    except Exception:
        df['Elevation_Binned'] = -1

    return downcast_df(df)

def compute_class_weights(y_arr, max_w=10.0):
    # y_arr expected 0..6 (may be missing some classes in subset)
    counts = np.bincount(y_arr, minlength=7).astype(np.float64)
    inv = np.zeros_like(counts)
    nonzero = counts > 0
    inv[nonzero] = 1.0 / counts[nonzero]
    inv[~nonzero] = 0.0
    # normalize to mean 1 over present classes
    if nonzero.any():
        inv = inv * (nonzero.sum() / inv[nonzero].sum())
    inv = np.clip(inv, 0.0, max_w)
    return inv

try:
    print("Loading data...", flush=True)
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    print(f"train shape: {train.shape}, test shape: {test.shape}")

    # Optional DEV subset
    SEED = 42
    DEV_N = int(os.environ.get('DEV_N', '200000'))
    if DEV_N and DEV_N < len(train):
        print(f"Subsampling train to {DEV_N} rows for fast dev...", flush=True)
        y_full = train['Cover_Type']
        if y_full.value_counts().min() >= 2:
            try:
                sss = StratifiedShuffleSplit(n_splits=1, test_size=len(train)-DEV_N, random_state=SEED)
                for keep_idx, _ in sss.split(train, y_full):
                    train = train.iloc[keep_idx].reset_index(drop=True)
                    break
            except Exception as e:
                print(f"Stratified subsample failed ({e}); falling back to random sample.")
                train = train.sample(n=DEV_N, random_state=SEED).reset_index(drop=True)
        else:
            train = train.sample(n=DEV_N, random_state=SEED).reset_index(drop=True)
        print(f"New train shape: {train.shape}")

    # Identify Id column if present
    id_col = None
    for c in ['Id', 'id', 'ID']:
        if c in train.columns:
            id_col = c; break

    target_col = 'Cover_Type'
    assert target_col in train.columns, 'Target Cover_Type not found'

    # Map target 1..7 -> 0..6
    y = (train[target_col].astype(int) - 1)
    print("y class counts:")
    print(y.value_counts().sort_index())

    print('Engineering & downcasting features...', flush=True)
    t_feat = time.time()
    train_fe = add_features(train.copy())
    gc.collect()
    test_fe = add_features(test.copy())
    gc.collect()
    print(f"Feature engineering done in {time.time()-t_feat:.1f}s", flush=True)

    # Build feature list
    drop_cols = [c for c in [target_col, id_col] if c is not None]
    features = [c for c in train_fe.columns if c not in drop_cols]
    print(f"Using {len(features)} features")

    X = train_fe[features]
    X_test = test_fe[features]

    # CV setup: only use StratifiedKFold if ALL 7 classes present in subset
    N_SPLITS = 3
    present_classes = np.unique(y.values)
    if len(present_classes) == 7 and (y.value_counts().min() >= N_SPLITS):
        print("Using StratifiedKFold", flush=True)
        splitter = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
        split_iter = splitter.split(X, y)
    else:
        print(f"Using KFold (unique classes in subset={len(present_classes)})", flush=True)
        splitter = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
        split_iter = splitter.split(X)

    # LightGBM CPU params (dev-fast)
    lgb_params = dict(
        objective='multiclass',
        num_class=7,
        metric='multi_logloss',
        learning_rate=0.08,
        num_leaves=256,
        max_depth=12,
        min_data_in_leaf=80,
        feature_fraction=0.9,
        bagging_fraction=0.9,
        bagging_freq=1,
        lambda_l1=0.0,
        lambda_l2=0.0,
        max_bin=255,
        n_estimators=2000,
        random_state=SEED,
        n_jobs=-1,
        force_row_wise=True
    )

    oof_probs = np.zeros((len(train), 7), dtype=np.float32)
    test_probs = np.zeros((len(test), 7), dtype=np.float32)
    fold_accuracies = []

    # Toggle weights for dev (disable to ensure splits form)
    USE_WEIGHTS = False
    cls_w = compute_class_weights(y.values, max_w=10.0) if USE_WEIGHTS else None
    if USE_WEIGHTS:
        print("Class weights:", {i: float(w) for i, w in enumerate(cls_w)})

    for fold, (tr_idx, va_idx) in enumerate(split_iter, 1):
        t0 = time.time()
        print(f"\n[Fold {fold}/{N_SPLITS}] train={len(tr_idx)} valid={len(va_idx)}", flush=True)
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        sw = (cls_w[y_tr.values] if USE_WEIGHTS else None)

        model = lgb.LGBMClassifier(**lgb_params)
        model.fit(
            X_tr, y_tr,
            sample_weight=sw,
            eval_set=[(X_va, y_va)],
            eval_metric='multi_logloss',
            callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=True)]
        )

        va_proba = model.predict_proba(X_va, raw_score=False)
        oof_probs[va_idx] = va_proba
        va_pred = np.argmax(va_proba, axis=1)
        acc = accuracy_score(y_va, va_pred)
        fold_accuracies.append(acc)
        print(f"Fold {fold} ACC: {acc:.6f} | best_iter: {model.best_iteration_} | elapsed: {time.time()-t0:.1f}s", flush=True)

        test_probs += model.predict_proba(X_test) / N_SPLITS

    oof_pred = np.argmax(oof_probs, axis=1)
    oof_acc = accuracy_score(y, oof_pred)
    print(f"\nOOF accuracy: {oof_acc:.6f}")
    print("Per-fold ACC:", ', '.join(f"{a:.6f}" for a in fold_accuracies))
    print(f"Total elapsed: {time.time()-t0_all:.1f}s")

    # Make submission
    sub = pd.DataFrame()
    sub['Id'] = test[id_col] if id_col is not None else np.arange(len(test))
    sub['Cover_Type'] = np.argmax(test_probs, axis=1) + 1  # back to 1..7
    sub_path = 'submission.csv'
    sub.to_csv(sub_path, index=False)
    print(f"Saved submission to {sub_path} with shape {sub.shape}")
except Exception as e:
    print("ERROR during run:", e)
    traceback.print_exc()

[ok] numpy already installed
[ok] pandas already installed
[ok] lightgbm already installed
Loading data...


train shape: (3600000, 56), test shape: (400000, 55)
Subsampling train to 200000 rows for fast dev...


New train shape: (200000, 56)
y class counts:
Cover_Type
0     73824
1    112812
2      9709
3        15
5       553
6      3087
Name: count, dtype: int64
Engineering & downcasting features...


Feature engineering done in 1.0s


Using 34 features
Using KFold (unique classes in subset=6)


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.85, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.85
[LightGBM] [Warning] lambda_l2 is set=0.2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] bagging_fraction is set=0.85, subsample=1.0 will be ignored. Current value: bagging_fraction=0.85


ERROR during run: shape mismatch: value array of shape (66667,6) could not be broadcast to indexing result of shape (66667,7)


Traceback (most recent call last):
  File "/tmp/ipykernel_237/2958152172.py", line 270, in <module>
    oof_probs[va_idx] = va_proba
    ~~~~~~~~~^^^^^^^^
ValueError: shape mismatch: value array of shape (66667,6) could not be broadcast to indexing result of shape (66667,7)


In [13]:
import os, gc, json, pandas as pd, numpy as np, psutil, shutil, time
print("Post-run probe: checking OOF metrics and submission file...", flush=True)

def show(var):
    if var in globals():
        val = globals()[var]
        try:
            if isinstance(val, (list, tuple, np.ndarray)):
                print(f"- {var}: type={type(val).__name__}, len/shape={len(val) if hasattr(val,'__len__') else 'NA'}/{getattr(val,'shape', 'NA')}")
            else:
                print(f"- {var}: {val}")
        except Exception:
            print(f"- {var}: type={type(val).__name__}")
        return True
    else:
        print(f"- {var}: <not in globals>")
        return False

checked = []
for v in ['oof_acc','fold_accuracies','features','N_SPLITS','SEED','train','test','y','oof_probs','test_probs']:
    checked.append(show(v))

if 'y' in globals():
    try:
        vc = y.value_counts().sort_index()
        print("y value_counts (sorted):\n", vc)
    except Exception as e:
        print("Could not show y value_counts:", e)

sub_path = 'submission.csv'
if os.path.exists(sub_path):
    sub = pd.read_csv(sub_path)
    print(f"submission.csv exists. shape={sub.shape}; Cover_Type head:")
    print(sub.head())
    print("Cover_Type distribution (top 10):\n", sub['Cover_Type'].value_counts().sort_index().head(10))
else:
    print("submission.csv not found.")

# Quick disk/mem stats
total, used, free = shutil.disk_usage('/')
print(f"Disk free: {free/1e9:.1f} GB of {total/1e9:.1f} GB")
mem = psutil.virtual_memory()
print(f"RAM used: {mem.used/1e9:.1f} GB / {mem.total/1e9:.1f} GB")
gc.collect()
print("Probe done.", flush=True)

Post-run probe: checking OOF metrics and submission file...


- oof_acc: 0.36912
- fold_accuracies: type=list, len/shape=0/NA
- features: type=list, len/shape=34/NA
- N_SPLITS: 3
- SEED: 42
- train:              Id  Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0        379246       2893     112      7                               250   
1         70624       3254      35      8                               133   
2       1699813       2598      32      3                               207   
3        258430       3260      90     12                               424   
4       2721920       3355     280     18                               143   
...         ...        ...     ...    ...                               ...   
199995  1817267       2705     102     16                               176   
199996   702296       2997     102     24                               205   
199997  1383413       3342     281     27                               354   
199998  1982758       3010     158     27                               1

In [10]:
import numpy as np, pandas as pd, os, time
print("Computing OOF accuracy from existing oof_probs/y and saving submission from test_probs...", flush=True)
t0 = time.time()
if 'oof_probs' in globals() and 'y' in globals():
    oof_pred = np.argmax(oof_probs, axis=1)
    # y is 0-based already
    from sklearn.metrics import accuracy_score
    oof_acc = accuracy_score(y, oof_pred)
    print(f"OOF accuracy (recomputed): {oof_acc:.6f}")
else:
    print("oof_probs or y missing; cannot compute OOF.")

sub_path = 'submission.csv'
if 'test_probs' in globals() and 'test' in globals():
    # Determine ID column
    id_col = None
    for c in ['Id','id','ID']:
        if c in test.columns:
            id_col = c; break
    sub = pd.DataFrame()
    sub['Id'] = test[id_col] if id_col is not None else np.arange(len(test_probs))
    sub['Cover_Type'] = np.argmax(test_probs, axis=1) + 1  # back to 1..7
    sub.to_csv(sub_path, index=False)
    print(f"Saved submission to {sub_path} with shape {sub.shape}")
else:
    print("test_probs or test missing; cannot write submission.")
print(f"Done in {time.time()-t0:.2f}s", flush=True)

Computing OOF accuracy from existing oof_probs/y and saving submission from test_probs...


OOF accuracy (recomputed): 0.369120


Saved submission to submission.csv with shape (400000, 2)
Done in 0.29s


In [19]:
import sys, subprocess, time, os, gc, traceback
t0 = time.time()
print("Setting up GPU XGBoost and running 3-fold CV on DEV subset (200k)...", flush=True)

def pip_install(pkg):
    try:
        __import__(pkg.split('==')[0])
        print(f"[ok] {pkg} present")
    except Exception:
        print(f"[install] {pkg}")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])

try:
    pip_install('xgboost==2.0.3')
    import numpy as np, pandas as pd, xgboost as xgb
    from sklearn.model_selection import KFold, StratifiedShuffleSplit
    from sklearn.metrics import accuracy_score

    # Load data fresh
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')

    # DEV subset 200k with stratified attempt else random
    SEED = 42
    DEV_N = int(os.environ.get('DEV_N', '200000'))
    y_full = train['Cover_Type']
    if DEV_N and DEV_N < len(train):
        if y_full.value_counts().min() >= 2:
            try:
                sss = StratifiedShuffleSplit(n_splits=1, test_size=len(train)-DEV_N, random_state=SEED)
                for keep_idx, _ in sss.split(train, y_full):
                    train = train.iloc[keep_idx].reset_index(drop=True)
                    break
            except Exception:
                train = train.sample(n=DEV_N, random_state=SEED).reset_index(drop=True)
        else:
            train = train.sample(n=DEV_N, random_state=SEED).reset_index(drop=True)

    id_col = 'Id' if 'Id' in train.columns else None
    y = (train['Cover_Type'].astype(int) - 1).astype(np.int32)

    # Self-contained feature engineering to avoid cross-cell function conflicts
    def fe(df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        Hhyd = 'Horizontal_Distance_To_Hydrology'; Vhyd = 'Vertical_Distance_To_Hydrology'
        hs9, hs12, hs3 = 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm'
        aspect = 'Aspect'; elev = 'Elevation'
        Hroad='Horizontal_Distance_To_Roadways'; Hfire='Horizontal_Distance_To_Fire_Points'

        if all(c in df.columns for c in [Hhyd, Vhyd]):
            df['Euclidean_Distance_To_Hydrology'] = np.sqrt(df[Hhyd]**2 + df[Vhyd]**2)
            df['Manhattan_Distance_To_Hydrology'] = np.abs(df[Hhyd]) + np.abs(df[Vhyd])
            df['Elevation_VD_Hydrology'] = df[elev] - df[Vhyd]
            df['Elevation_Plus_VD_Hydrology'] = df[elev] + df[Vhyd]
            df['Elevation_minus_Euclidean_Dist_Hydrology'] = df[elev] - df['Euclidean_Distance_To_Hydrology']
            df['Hydro_Ratio'] = df[Hhyd] / (df[Vhyd].abs() + 1.0)
            ang = np.arctan2(df[Vhyd].astype(float), df[Hhyd].astype(float))
            df['Hydro_Angle_Sin'] = np.sin(ang); df['Hydro_Angle_Cos'] = np.cos(ang)

        if all(c in df.columns for c in [hs9, hs12, hs3]):
            df['Hillshade_Mean'] = (df[hs9] + df[hs12] + df[hs3]) / 3.0
            df['Hillshade_Min'] = df[[hs9, hs12, hs3]].min(axis=1)
            df['Hillshade_Max'] = df[[hs9, hs12, hs3]].max(axis=1)
            df['Hillshade_Range'] = df['Hillshade_Max'] - df['Hillshade_Min']
            df['Hillshade_Diff_9_3'] = df[hs9] - df[hs3]

        if aspect in df.columns:
            rad = np.deg2rad(df[aspect].astype(float))
            df['Aspect_Sin'] = np.sin(rad); df['Aspect_Cos'] = np.cos(rad)

        if Hroad in df.columns and Hfire in df.columns:
            df['Road_Fire_AbsDiff'] = np.abs(df[Hroad] - df[Hfire])
        if Hhyd in df.columns and Hroad in df.columns:
            df['Hydro_Road_AbsDiff'] = np.abs(df[Hhyd] - df[Hroad])
        if Hhyd in df.columns and Hfire in df.columns:
            df['Hydro_Fire_AbsDiff'] = np.abs(df[Hhyd] - df[Hfire])

        # Compress Wilderness_Area one-hots (lexicographic order is fine for argmax index)
        w_cols = [c for c in df.columns if c.startswith('Wilderness_Area')]
        if w_cols:
            w_cols_sorted = sorted(w_cols)
            warr = df[w_cols_sorted].to_numpy(dtype=np.int8, copy=False)
            w_cat = warr.argmax(axis=1).astype(np.int16) + 1
            df['Wilderness_Area_cat'] = w_cat
            df.drop(columns=w_cols_sorted, inplace=True)
        else:
            df['Wilderness_Area_cat'] = 0

        # Compress Soil_Type one-hots
        s_cols = [c for c in df.columns if c.startswith('Soil_Type')]
        if s_cols:
            s_cols_sorted = sorted(s_cols)
            sarr = df[s_cols_sorted].to_numpy(dtype=np.int8, copy=False)
            s_cat = sarr.argmax(axis=1).astype(np.int16) + 1
            df['Soil_Type_cat'] = s_cat
            df.drop(columns=s_cols_sorted, inplace=True)
        else:
            df['Soil_Type_cat'] = 0

        # Interaction and binned elevation
        df['Soil_Wilderness_Interaction'] = (df['Soil_Type_cat'].astype(np.int32)*100 + df['Wilderness_Area_cat'].astype(np.int32)).astype(np.int32)
        try:
            df['Elevation_Binned'] = pd.cut(df[elev], bins=30, labels=False).astype('float32').fillna(-1).astype('int16')
        except Exception:
            df['Elevation_Binned'] = -1

        # Downcast numerics to float32 for XGB
        for c in df.columns:
            if c == 'Cover_Type' or c == 'Id':
                continue
            if pd.api.types.is_float_dtype(df[c]) or pd.api.types.is_integer_dtype(df[c]):
                df[c] = df[c].astype(np.float32)
        return df

    train_fe = fe(train)
    test_fe = fe(test)

    # Build feature list (drop Id and target)
    drop_cols = [c for c in ['Cover_Type', id_col] if c is not None]
    features = [c for c in train_fe.columns if c not in drop_cols]
    X = train_fe[features].astype(np.float32)
    X_test = test_fe[features].astype(np.float32)

    print(f"X shape: {X.shape}, X_test: {X_test.shape}, features: {len(features)}", flush=True)

    # XGBoost params (GPU)
    xgb_params = {
        'objective': 'multi:softprob',
        'num_class': 7,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'learning_rate': 0.06,
        'max_depth': 8,
        'min_child_weight': 8,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'lambda': 1.0,
        'alpha': 0.0,
        'eval_metric': 'mlogloss'
    }

    N_SPLITS = 3
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    oof = np.zeros((len(X), 7), dtype=np.float32)
    test_pred = np.zeros((len(X_test), 7), dtype=np.float32)
    fold_acc = []

    for i, (tr, va) in enumerate(kf.split(X), 1):
        t_fold = time.time()
        print(f"[XGB Fold {i}/{N_SPLITS}] train={len(tr)} valid={len(va)}", flush=True)
        dtr = xgb.DMatrix(X.iloc[tr], label=y.iloc[tr])
        dva = xgb.DMatrix(X.iloc[va], label=y.iloc[va])
        dte = xgb.DMatrix(X_test)
        booster = xgb.train(
            params=xgb_params,
            dtrain=dtr,
            num_boost_round=2500,
            evals=[(dtr,'train'), (dva,'valid')],
            early_stopping_rounds=100,
            verbose_eval=200
        )
        p_va = booster.predict(dva, iteration_range=(0, booster.best_iteration+1))
        oof[va] = p_va
        p_te = booster.predict(dte, iteration_range=(0, booster.best_iteration+1))
        test_pred += p_te / N_SPLITS
        acc = accuracy_score(y.iloc[va], np.argmax(p_va, axis=1))
        fold_acc.append(acc)
        print(f"Fold {i} ACC={acc:.6f} | iters={booster.best_iteration+1} | elapsed={time.time()-t_fold:.1f}s", flush=True)
        del dtr, dva, dte, booster; gc.collect()

    oof_acc = accuracy_score(y, np.argmax(oof, axis=1))
    print(f"XGB OOF ACC: {oof_acc:.6f}; per-fold: {', '.join(f'{a:.6f}' for a in fold_acc)}")

    sub = pd.DataFrame({'Id': test['Id'] if 'Id' in test.columns else np.arange(len(test_pred)),
                        'Cover_Type': np.argmax(test_pred, axis=1) + 1})
    sub.to_csv('submission.csv', index=False)
    print(f"Saved submission.csv with shape {sub.shape}")

except Exception as e:
    print('ERROR in XGB cell:', e)
    traceback.print_exc()
print(f"Done in {time.time()-t0:.1f}s")

Setting up GPU XGBoost and running 3-fold CV on DEV subset (200k)...


[ok] xgboost==2.0.3 present


X shape: (200000, 32), X_test: (400000, 32), features: 32


[XGB Fold 1/3] train=133333 valid=66667


[0]	train-mlogloss:1.76225	valid-mlogloss:1.76322


...

[345]	train-mlogloss:0.05213	valid-mlogloss:0.11755


Fold 3 ACC=0.950875 | iters=246 | elapsed=6.7s


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:44:06] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


XGB OOF ACC: 0.950510; per-fold: 0.949240, 0.951415, 0.950875


Saved submission.csv with shape (400000, 2)
Done in 34.1s


In [25]:
import sys, subprocess, time, os, gc, traceback
t0 = time.time()
print("Setting up GPU CatBoost and running 3-fold CV on DEV subset (200k)...", flush=True)

def pip_install(pkg):
    try:
        __import__(pkg.split('==')[0])
        print(f"[ok] {pkg} present")
    except Exception:
        print(f"[install] {pkg}")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])

try:
    pip_install('catboost==1.2.5')
    import numpy as np, pandas as pd
    from catboost import CatBoostClassifier, Pool
    from sklearn.model_selection import KFold, StratifiedShuffleSplit
    from sklearn.metrics import accuracy_score

    # Load data fresh
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')

    # DEV subset 200k with stratified attempt else random
    SEED = 42
    DEV_N = int(os.environ.get('DEV_N', '200000'))
    y_full = train['Cover_Type']
    if DEV_N and DEV_N < len(train):
        if y_full.value_counts().min() >= 2:
            try:
                sss = StratifiedShuffleSplit(n_splits=1, test_size=len(train)-DEV_N, random_state=SEED)
                for keep_idx, _ in sss.split(train, y_full):
                    train = train.iloc[keep_idx].reset_index(drop=True)
                    break
            except Exception:
                train = train.sample(n=DEV_N, random_state=SEED).reset_index(drop=True)
        else:
            train = train.sample(n=DEV_N, random_state=SEED).reset_index(drop=True)

    id_col = 'Id' if 'Id' in train.columns else None
    y = (train['Cover_Type'].astype(int) - 1).astype(np.int32)

    # Self-contained FE (same as XGB cell) to avoid cross-cell deps
    def fe(df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        Hhyd = 'Horizontal_Distance_To_Hydrology'; Vhyd = 'Vertical_Distance_To_Hydrology'
        hs9, hs12, hs3 = 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm'
        aspect = 'Aspect'; elev = 'Elevation'
        Hroad='Horizontal_Distance_To_Roadways'; Hfire='Horizontal_Distance_To_Fire_Points'

        if all(c in df.columns for c in [Hhyd, Vhyd]):
            df['Euclidean_Distance_To_Hydrology'] = np.sqrt(df[Hhyd]**2 + df[Vhyd]**2)
            df['Manhattan_Distance_To_Hydrology'] = np.abs(df[Hhyd]) + np.abs(df[Vhyd])
            df['Elevation_VD_Hydrology'] = df[elev] - df[Vhyd]
            df['Elevation_Plus_VD_Hydrology'] = df[elev] + df[Vhyd]
            df['Elevation_minus_Euclidean_Dist_Hydrology'] = df[elev] - df['Euclidean_Distance_To_Hydrology']
            df['Hydro_Ratio'] = df[Hhyd] / (df[Vhyd].abs() + 1.0)
            ang = np.arctan2(df[Vhyd].astype(float), df[Hhyd].astype(float))
            df['Hydro_Angle_Sin'] = np.sin(ang); df['Hydro_Angle_Cos'] = np.cos(ang)

        if all(c in df.columns for c in [hs9, hs12, hs3]):
            df['Hillshade_Mean'] = (df[hs9] + df[hs12] + df[hs3]) / 3.0
            df['Hillshade_Min'] = df[[hs9, hs12, hs3]].min(axis=1)
            df['Hillshade_Max'] = df[[hs9, hs12, hs3]].max(axis=1)
            df['Hillshade_Range'] = df['Hillshade_Max'] - df['Hillshade_Min']
            df['Hillshade_Diff_9_3'] = df[hs9] - df[hs3]

        if aspect in df.columns:
            rad = np.deg2rad(df[aspect].astype(float))
            df['Aspect_Sin'] = np.sin(rad); df['Aspect_Cos'] = np.cos(rad)

        if Hroad in df.columns and Hfire in df.columns:
            df['Road_Fire_AbsDiff'] = np.abs(df[Hroad] - df[Hfire])
        if Hhyd in df.columns and Hroad in df.columns:
            df['Hydro_Road_AbsDiff'] = np.abs(df[Hhyd] - df[Hroad])
        if Hhyd in df.columns and Hfire in df.columns:
            df['Hydro_Fire_AbsDiff'] = np.abs(df[Hhyd] - df[Hfire])

        # Compress Wilderness_Area one-hots (lexicographic order)
        w_cols = [c for c in df.columns if c.startswith('Wilderness_Area')]
        if w_cols:
            w_cols_sorted = sorted(w_cols)
            warr = df[w_cols_sorted].to_numpy(dtype=np.int8, copy=False)
            w_cat = warr.argmax(axis=1).astype(np.int16) + 1
            df['Wilderness_Area_cat'] = w_cat
            df.drop(columns=w_cols_sorted, inplace=True)
        else:
            df['Wilderness_Area_cat'] = 0

        # Compress Soil_Type one-hots (lexicographic order)
        s_cols = [c for c in df.columns if c.startswith('Soil_Type')]
        if s_cols:
            s_cols_sorted = sorted(s_cols)
            sarr = df[s_cols_sorted].to_numpy(dtype=np.int8, copy=False)
            s_cat = sarr.argmax(axis=1).astype(np.int16) + 1
            df['Soil_Type_cat'] = s_cat
            df.drop(columns=s_cols_sorted, inplace=True)
        else:
            df['Soil_Type_cat'] = 0

        # Interaction and binned elevation
        df['Soil_Wilderness_Interaction'] = (df['Soil_Type_cat'].astype(np.int32)*100 + df['Wilderness_Area_cat'].astype(np.int32)).astype(np.int32)
        try:
            df['Elevation_Binned'] = pd.cut(df[elev], bins=30, labels=False).astype('float32').fillna(-1).astype('int16')
        except Exception:
            df['Elevation_Binned'] = -1

        # Cast numerics to float32 for CatBoost too (it can handle float features well)
        for c in df.columns:
            if c == 'Cover_Type' or c == 'Id':
                continue
            if pd.api.types.is_float_dtype(df[c]) or pd.api.types.is_integer_dtype(df[c]):
                df[c] = df[c].astype(np.float32)
        return df

    train_fe = fe(train)
    test_fe = fe(test)

    # Build feature list (drop Id and target)
    drop_cols = [c for c in ['Cover_Type', id_col] if c is not None]
    features = [c for c in train_fe.columns if c not in drop_cols]
    X = train_fe[features].astype(np.float32)
    X_test = test_fe[features].astype(np.float32)
    print(f"X shape: {X.shape}, X_test: {X_test.shape}, features: {len(features)}", flush=True)

    # CatBoost params (GPU)
    cat_params = {
        'loss_function': 'MultiClass',
        'task_type': 'GPU',
        'devices': '0',
        'iterations': 5000,
        'learning_rate': 0.06,
        'depth': 8,
        'l2_leaf_reg': 4.0,
        'border_count': 254,
        'random_strength': 0.8,
        'bagging_temperature': 0.7,
        'od_type': 'Iter',
        'od_wait': 200,
        'verbose': 200,
        'random_seed': SEED,
        'classes_count': 7
    }

    def to_full_proba(p: np.ndarray, present_classes: np.ndarray, n_classes: int = 7) -> np.ndarray:
        # If CatBoost already outputs n_classes columns, return as-is; otherwise expand by present indices
        if p.shape[1] == n_classes:
            return p.astype(np.float32, copy=False)
        full = np.zeros((p.shape[0], n_classes), dtype=np.float32)
        full[:, present_classes] = p
        return full

    N_SPLITS = 3
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    oof_cat = np.zeros((len(X), 7), dtype=np.float32)
    test_pred_cat = np.zeros((len(X_test), 7), dtype=np.float32)
    fold_acc = []

    for i, (tr, va) in enumerate(kf.split(X), 1):
        t_fold = time.time()
        print(f"[CAT Fold {i}/{N_SPLITS}] train={len(tr)} valid={len(va)}", flush=True)
        tr_pool = Pool(X.iloc[tr], label=y.iloc[tr])
        va_pool = Pool(X.iloc[va], label=y.iloc[va])
        model = CatBoostClassifier(**cat_params)
        model.fit(tr_pool, eval_set=va_pool, use_best_model=True)
        present = np.unique(y.iloc[tr].values)
        p_va = model.predict_proba(va_pool)
        p_va_full = to_full_proba(p_va, present, n_classes=7)
        oof_cat[va] = p_va_full
        acc = accuracy_score(y.iloc[va], np.argmax(p_va_full, axis=1))
        fold_acc.append(acc)
        p_te = model.predict_proba(X_test)
        p_te_full = to_full_proba(p_te, present, n_classes=7)
        test_pred_cat += p_te_full / N_SPLITS
        print(f"Fold {i} ACC={acc:.6f} | best_tree={model.get_best_iteration()} | elapsed={time.time()-t_fold:.1f}s", flush=True)
        del tr_pool, va_pool, model; gc.collect()

    oof_acc_cat = accuracy_score(y, np.argmax(oof_cat, axis=1))
    print(f"CAT OOF ACC: {oof_acc_cat:.6f}; per-fold: {', '.join(f'{a:.6f}' for a in fold_acc)}")

    # Save a CatBoost-only submission preview (will be overwritten by ensemble later)
    sub_cat = pd.DataFrame({'Id': test['Id'] if 'Id' in test.columns else np.arange(len(test_pred_cat)),
                            'Cover_Type': np.argmax(test_pred_cat, axis=1) + 1})
    sub_cat.to_csv('submission_cat.csv', index=False)
    print(f"Saved submission_cat.csv with shape {sub_cat.shape}")

    # If XGB preds available (from previous cell), create a simple ensemble and write submission.csv
    if 'test_pred' in globals():
        # test_pred from XGB cell; oof available as 'oof'
        w_xgb, w_cat = 0.6, 0.4
        ens_test = w_xgb * test_pred + w_cat * test_pred_cat
        sub = pd.DataFrame({'Id': test['Id'] if 'Id' in test.columns else np.arange(len(ens_test)),
                            'Cover_Type': np.argmax(ens_test, axis=1) + 1})
        sub.to_csv('submission.csv', index=False)
        print(f"Saved ensemble submission.csv (XGB {w_xgb:.2f} + CAT {w_cat:.2f}) with shape {sub.shape}")
    else:
        print("XGB predictions not found in kernel; keeping submission_cat.csv only.")

except Exception as e:
    print('ERROR in CatBoost cell:', e)
    traceback.print_exc()
print(f"Done in {time.time()-t0:.1f}s")

Setting up GPU CatBoost and running 3-fold CV on DEV subset (200k)...


[ok] catboost==1.2.5 present


X shape: (200000, 32), X_test: (400000, 32), features: 32


[CAT Fold 1/3] train=133333 valid=66667


Found only 6 unique classes in the data, but have defined 7 classes. Probably something is wrong with data.


...

bestTest = 0.1186861893
bestIteration = 1771
Shrink model to first 1772 iterations.


Fold 3 ACC=0.950395 | best_tree=1771 | elapsed=24.4s


CAT OOF ACC: 0.950385; per-fold: 0.949675, 0.951085, 0.950395


Saved submission_cat.csv with shape (400000, 2)


Saved ensemble submission.csv (XGB 0.60 + CAT 0.40) with shape (400000, 2)
Done in 85.9s


In [29]:
import sys, subprocess, time, os, gc, traceback
print("Full-data 5-fold training: XGBoost (GPU) + CatBoost (GPU) with enhanced FE and OOF weight optimization", flush=True)

def ensure(pkg):
    try:
        __import__(pkg.split('==')[0])
        print(f"[ok] {pkg} present")
    except Exception:
        print(f"[install] {pkg}")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])

t0_all = time.time()
try:
    ensure('xgboost==2.0.3')
    ensure('catboost==1.2.5')
    import numpy as np, pandas as pd, xgboost as xgb
    from catboost import CatBoostClassifier, Pool
    from sklearn.model_selection import KFold
    from sklearn.metrics import accuracy_score

    SEED = int(os.environ.get('SEED','42'))
    N_SPLITS = 5
    np.random.seed(SEED)

    print("Loading data...", flush=True)
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    id_col = 'Id' if 'Id' in train.columns else None
    y = (train['Cover_Type'].astype(int) - 1).astype(np.int32)
    print(f"train: {train.shape}, test: {test.shape}")

    # Enhanced FE (keep one-hots; add compressed cats and additional interactions)
    def fe(df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        Hhyd = 'Horizontal_Distance_To_Hydrology'; Vhyd = 'Vertical_Distance_To_Hydrology'
        hs9, hs12, hs3 = 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm'
        aspect = 'Aspect'; elev = 'Elevation'
        Hroad = 'Horizontal_Distance_To_Roadways'; Hfire = 'Horizontal_Distance_To_Fire_Points'

        # Core hydrology distances/angles
        df['Euclidean_Distance_To_Hydrology'] = np.sqrt(df[Hhyd]**2 + df[Vhyd]**2)
        df['Manhattan_Distance_To_Hydrology'] = np.abs(df[Hhyd]) + np.abs(df[Vhyd])
        df['Elevation_VD_Hydrology'] = df[elev] - df[Vhyd]
        df['Elevation_Plus_VD_Hydrology'] = df[elev] + df[Vhyd]
        df['Elevation_minus_Euclidean_Dist_Hydrology'] = df[elev] - df['Euclidean_Distance_To_Hydrology']
        df['Hydro_Ratio'] = df[Hhyd] / (df[Vhyd].abs() + 1.0)
        ang = np.arctan2(df[Vhyd].astype(float), df[Hhyd].astype(float))
        df['Hydro_Angle_Sin'] = np.sin(ang); df['Hydro_Angle_Cos'] = np.cos(ang)

        # Hillshade statistics
        df['Hillshade_Mean'] = (df[hs9] + df[hs12] + df[hs3]) / 3.0
        df['Hillshade_Min'] = df[[hs9, hs12, hs3]].min(axis=1)
        df['Hillshade_Max'] = df[[hs9, hs12, hs3]].max(axis=1)
        df['Hillshade_Range'] = df['Hillshade_Max'] - df['Hillshade_Min']
        df['Hillshade_Diff_9_3'] = df[hs9] - df[hs3]

        # Aspect transforms
        rad = np.deg2rad(df[aspect].astype(float))
        df['Aspect_Sin'] = np.sin(rad); df['Aspect_Cos'] = np.cos(rad)

        # Distance interactions
        df['Road_Fire_AbsDiff'] = np.abs(df[Hroad] - df[Hfire])
        df['Hydro_Road_AbsDiff'] = np.abs(df[Hhyd] - df[Hroad])
        df['Hydro_Fire_AbsDiff'] = np.abs(df[Hhyd] - df[Hfire])

        # Expert additions
        df['Slope_Elevation_Product'] = df['Slope'] * df['Elevation']
        df['Elevation_div_Slope'] = df['Elevation'] / (df['Slope'] + 1.0)
        df['Hillshade_Std'] = df[[hs9, hs12, hs3]].std(axis=1)
        df['HS_9_over_Noon'] = df[hs9] / (df[hs12] + 1.0)
        df['Elevation_minus_H_Roads'] = df['Elevation'] - df[Hroad]
        df['Total_Distance_Sum'] = df[Hhyd] + df[Hroad] + df[Hfire]
        df['VHyd_Neg'] = (df[Vhyd] < 0).astype(np.int16)

        # Keep one-hots, also add compressed categories and sums
        w_cols = [c for c in df.columns if c.startswith('Wilderness_Area')]
        if w_cols:
            w_cols_sorted = sorted(w_cols)
            warr = df[w_cols_sorted].to_numpy(dtype=np.int8, copy=False)
            w_cat = warr.argmax(axis=1).astype(np.int16) + 1
            df['Wilderness_Area_cat'] = w_cat
            df['Wilderness_Area_Sum'] = warr.sum(axis=1).astype(np.int16)
        else:
            df['Wilderness_Area_cat'] = 0; df['Wilderness_Area_Sum'] = 0
        s_cols = [c for c in df.columns if c.startswith('Soil_Type')]
        if s_cols:
            s_cols_sorted = sorted(s_cols)
            sarr = df[s_cols_sorted].to_numpy(dtype=np.int8, copy=False)
            s_cat = sarr.argmax(axis=1).astype(np.int16) + 1
            df['Soil_Type_cat'] = s_cat
            df['Soil_Type_Sum'] = sarr.sum(axis=1).astype(np.int16)
        else:
            df['Soil_Type_cat'] = 0; df['Soil_Type_Sum'] = 0
        df['Soil_Wilderness_Interaction'] = (df['Soil_Type_cat'].astype(np.int32)*100 + df['Wilderness_Area_cat'].astype(np.int32)).astype(np.int32)
        # Elevation binned
        try:
            df['Elevation_Binned'] = pd.cut(df['Elevation'], bins=30, labels=False).astype('float32').fillna(-1).astype('int16')
        except Exception:
            df['Elevation_Binned'] = -1

        # Downcast numerics
        for c in df.columns:
            if c == 'Cover_Type':
                continue
            if pd.api.types.is_integer_dtype(df[c]):
                df[c] = pd.to_numeric(df[c], downcast='integer')
            elif pd.api.types.is_float_dtype(df[c]):
                df[c] = pd.to_numeric(df[c], downcast='float')
        return df

    t_feat = time.time()
    train_fe = fe(train)
    test_fe = fe(test)
    print(f"FE done in {time.time()-t_feat:.1f}s", flush=True)

    # Features (keep one-hots); drop Id and target
    drop_cols = ['Cover_Type'] + ([id_col] if id_col is not None else [])
    features = [c for c in train_fe.columns if c not in drop_cols]
    X = train_fe[features].astype(np.float32)
    X_test = test_fe[features].astype(np.float32)

    # Sanitize non-finite values
    def sanitize(df_: pd.DataFrame, name: str):
        a = df_.to_numpy()
        n_inf = np.isinf(a).sum()
        n_nan = np.isnan(a).sum()
        if n_inf or n_nan:
            print(f"Before sanitize {name}: inf={n_inf}, nan={n_nan}", flush=True)
        df_.replace([np.inf, -np.inf], np.nan, inplace=True)
        a = df_.to_numpy()
        n_inf2 = np.isinf(a).sum()
        n_nan2 = np.isnan(a).sum()
        if n_inf2 or n_nan2:
            print(f"After replace {name}: inf={n_inf2}, nan={n_nan2}", flush=True)
    sanitize(X, 'X')
    sanitize(X_test, 'X_test')
    print(f"Using {len(features)} features; X={X.shape}, X_test={X_test.shape}")

    # Common CV
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

    # XGBoost params (expert)
    xgb_params = {
        'objective': 'multi:softprob',
        'num_class': 7,
        'device': 'cuda',
        'tree_method': 'hist',
        'learning_rate': 0.03,
        'max_depth': 9,
        'min_child_weight': 5,
        'subsample': 0.8,
        'colsample_bytree': 0.75,
        'lambda': 1.5,
        'alpha': 0.5,
        'eval_metric': 'mlogloss'
    }

    # CatBoost params (expert)
    cat_params = {
        'loss_function': 'MultiClass',
        'task_type': 'GPU',
        'iterations': 8000,
        'learning_rate': 0.035,
        'depth': 9,
        'l2_leaf_reg': 5.0,
        'border_count': 254,
        'random_strength': 0.8,
        'bagging_temperature': 0.7,
        'od_type': 'Iter',
        'od_wait': 250,
        'classes_count': 7,
        'random_seed': SEED,
        'verbose': 200
    }

    # Storage
    oof_xgb = np.zeros((len(X), 7), dtype=np.float32)
    oof_cat = np.zeros((len(X), 7), dtype=np.float32)
    test_xgb = np.zeros((len(X_test), 7), dtype=np.float32)
    test_cat = np.zeros((len(X_test), 7), dtype=np.float32)
    acc_xgb, acc_cat = [], []

    # Train per fold
    for fold, (tr, va) in enumerate(kf.split(X), 1):
        t_fold = time.time()
        print(f"\n[Fold {fold}/{N_SPLITS}] train={len(tr)} valid={len(va)}", flush=True)
        # XGB
        dtr = xgb.DMatrix(X.iloc[tr], label=y.iloc[tr], missing=np.nan)
        dva = xgb.DMatrix(X.iloc[va], label=y.iloc[va], missing=np.nan)
        dte = xgb.DMatrix(X_test, missing=np.nan)
        booster = xgb.train(
            params=xgb_params,
            dtrain=dtr,
            num_boost_round=5000,
            evals=[(dtr,'train'), (dva,'valid')],
            early_stopping_rounds=200,
            verbose_eval=250
        )
        p_va = booster.predict(dva, iteration_range=(0, booster.best_iteration+1))
        oof_xgb[va] = p_va
        p_te = booster.predict(dte, iteration_range=(0, booster.best_iteration+1))
        test_xgb += p_te / N_SPLITS
        ax = accuracy_score(y.iloc[va], np.argmax(p_va, axis=1)); acc_xgb.append(ax)
        print(f"XGB fold ACC={ax:.6f}; iters={booster.best_iteration+1}")
        del dtr, dva, dte, booster; gc.collect()

        # CatBoost
        tr_pool = Pool(X.iloc[tr], label=y.iloc[tr])
        va_pool = Pool(X.iloc[va], label=y.iloc[va])
        model = CatBoostClassifier(**cat_params)
        model.fit(tr_pool, eval_set=va_pool, use_best_model=True)
        p_va = model.predict_proba(va_pool)
        oof_cat[va] = p_va.astype(np.float32)
        p_te = model.predict_proba(X_test)
        test_cat += p_te.astype(np.float32) / N_SPLITS
        ac = accuracy_score(y.iloc[va], np.argmax(p_va, axis=1)); acc_cat.append(ac)
        print(f"CAT fold ACC={ac:.6f}; best_iter={model.get_best_iteration()}")
        del tr_pool, va_pool, model; gc.collect()
        print(f"Fold {fold} elapsed {time.time()-t_fold:.1f}s", flush=True)

    # OOF metrics
    oof_acc_xgb = accuracy_score(y, np.argmax(oof_xgb, axis=1))
    oof_acc_cat = accuracy_score(y, np.argmax(oof_cat, axis=1))
    print(f"\nOOF XGB: {oof_acc_xgb:.6f}; per-fold: {', '.join(f'{a:.6f}' for a in acc_xgb)}")
    print(f"OOF CAT: {oof_acc_cat:.6f}; per-fold: {', '.join(f'{a:.6f}' for a in acc_cat)}")

    # Optimize blend weight on OOF
    def blend_acc(w):
        p = w * oof_xgb + (1.0 - w) * oof_cat
        return accuracy_score(y, np.argmax(p, axis=1))
    best_w, best_acc = 0.5, 0.0
    for w in np.linspace(0.3, 0.7, 41):
        a = blend_acc(float(w))
        if a > best_acc:
            best_acc, best_w = a, float(w)
    print(f"Best OOF blend: w_xgb={best_w:.2f}, acc={best_acc:.6f}")

    # Blend test
    ens_test = best_w * test_xgb + (1.0 - best_w) * test_cat
    sub = pd.DataFrame({'Id': test[id_col] if id_col in test.columns else np.arange(len(ens_test)),
                        'Cover_Type': np.argmax(ens_test, axis=1) + 1})
    sub.to_csv('submission.csv', index=False)
    # Also save model-wise submissions for debugging
    pd.DataFrame({'Id': test[id_col] if id_col in test.columns else np.arange(len(test_xgb)),
                  'Cover_Type': np.argmax(test_xgb, axis=1) + 1}).to_csv('submission_xgb.csv', index=False)
    pd.DataFrame({'Id': test[id_col] if id_col in test.columns else np.arange(len(test_cat)),
                  'Cover_Type': np.argmax(test_cat, axis=1) + 1}).to_csv('submission_cat_full.csv', index=False)
    print(f"Saved submission.csv (blended) with shape {sub.shape}")
    print(f"Total elapsed: {time.time()-t0_all:.1f}s")

except Exception as e:
    print('ERROR in full-data training cell:', e)
    traceback.print_exc()

Full-data 5-fold training: XGBoost (GPU) + CatBoost (GPU) with enhanced FE and OOF weight optimization


[ok] xgboost==2.0.3 present
[ok] catboost==1.2.5 present
Loading data...


train: (3600000, 56), test: (400000, 55)


FE done in 12.3s


Before sanitize X: inf=5911, nan=0


...

7999:	learn: 0.0584976	test: 0.0762700	best: 0.0762689 (7975)	total: 9m 7s	remaining: 0us
bestTest = 0.07626885851
bestIteration = 7975
Shrink model to first 7976 iterations.


CAT fold ACC=0.962478; best_iter=7975
Fold 5 elapsed 2815.0s



OOF XGB: 0.962498; per-fold: 0.962311, 0.962483, 0.962624, 0.962311, 0.962760
OOF CAT: 0.962419; per-fold: 0.962121, 0.962536, 0.962640, 0.962318, 0.962478


Best OOF blend: w_xgb=0.57, acc=0.962619


Saved submission.csv (blended) with shape (400000, 2)
Total elapsed: 12803.6s
